# Polygon IO Toolkit and Tools

This notebook shows how to use agents to interact with the [Polygon IO](https://polygon.io/) toolkit. The toolkit provides access to Polygon's Stock Market Data API.

## Setup

### Installation

To use Polygon IO tools, you need to install the `langchain-community` package.

In [ ]:
%pip install -qU langchain-community > /dev/null

### Credentials

Get your Polygon IO API key [here](https://polygon.io/), and then set it below. 

In [2]:
import getpass
import os

if "POLYGON_API_KEY" not in os.environ:
    os.environ["POLYGON_API_KEY"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Toolkit

Polygon IO provides both a toolkit and individual tools for each of the tools included in the toolkit. Let's first explore using the toolkit and then we will walk through using the individual tools.

### Initialization

We can initialize the toolkit by importing it alongside the API wrapper needed to use the tools.

In [4]:
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper

polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)

### Tools

We can examine the tools included in this toolkit:

In [5]:
toolkit.get_tools()

[PolygonAggregates(api_wrapper=PolygonAPIWrapper(polygon_api_key='K231Rg76pOsN_IdT2eWhCC9npBguCYEj')),
 PolygonLastQuote(api_wrapper=PolygonAPIWrapper(polygon_api_key='K231Rg76pOsN_IdT2eWhCC9npBguCYEj')),
 PolygonTickerNews(api_wrapper=PolygonAPIWrapper(polygon_api_key='K231Rg76pOsN_IdT2eWhCC9npBguCYEj')),
 PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='K231Rg76pOsN_IdT2eWhCC9npBguCYEj'))]

### Use within an agent

Next we can add our toolkit to an agent and use it!

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)

In [12]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

We can examine yesterdays information for a certain ticker:

In [13]:
agent_executor.invoke({"input": "What was yesterdays financial info for AAPL?"})



> Entering new AgentExecutor chain...

Invoking: `polygon_aggregates` with `{'ticker': 'AAPL', 'timespan': 'day', 'timespan_multiplier': 1, 'from_date': '2023-10-05', 'to_date': '2023-10-05'}`


[{"v": 48527918.0, "vw": 174.2297, "o": 173.79, "c": 174.91, "h": 175.45, "l": 172.68, "t": 1696478400000, "n": 509175}]Here is the financial information for Apple Inc. (AAPL) for October 5, 2023:

- **Opening Price**: $173.79
- **Closing Price**: $174.91
- **High Price**: $175.45
- **Low Price**: $172.68
- **Volume**: 48,527,918 shares
- **Volume Weighted Average Price (VWAP)**: $174.2297
- **Number of Transactions**: 509,175

> Finished chain.


{'input': 'What was yesterdays financial info for AAPL?',
 'output': 'Here is the financial information for Apple Inc. (AAPL) for October 5, 2023:\n\n- **Opening Price**: $173.79\n- **Closing Price**: $174.91\n- **High Price**: $175.45\n- **Low Price**: $172.68\n- **Volume**: 48,527,918 shares\n- **Volume Weighted Average Price (VWAP)**: $174.2297\n- **Number of Transactions**: 509,175'}

We can also ask for recent news regarding a stock:

In [14]:
agent_executor.invoke({"input": "What is the recent new regarding MSFT?"})



> Entering new AgentExecutor chain...

Invoking: `polygon_ticker_news` with `{'query': 'MSFT'}`


[{"id": "931602cbcfeb06e22188e205a1cb6127215b7a62e9a37e5cc7a935e8376ac402", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "Prediction: 1 Unstoppable Stock Will Join Nvidia, Apple, Microsoft, and Alphabet in the $2 Trillion Club Within 3 Years", "author": "The Motley Fool", "published_utc": "2024-08-08T10:40:00Z", "article_url": "https://www.fool.com/investing/2024/08/08/prediction-1-stock-nvidia-microsoft-in-2-trillion/?source=iedfolrf0000001", "tickers": ["META", "NVDA", "AAPL", "MSFT", "GOOG", "GOOGL"], "image_url": "https://g.foolcdn.com/editorial/images/785914/two-people-laughing-while-watching-a-video-on-a-smartphone.jpg", "description": "Meta Platforms (META) is predicted t

{'input': 'What is the recent new regarding MSFT?',
 'output': 'Here are some recent news articles regarding Microsoft (MSFT):\n\n1. **Prediction: 1 Unstoppable Stock Will Join Nvidia, Apple, Microsoft, and Alphabet in the $2 Trillion Club Within 3 Years**\n   - **Publisher:** The Motley Fool\n   - **Published Date:** August 8, 2024\n   - **Summary:** The article predicts that Meta Platforms (META) will join the $2 trillion club within 3 years, driven by its advancements in artificial intelligence and potential to drive significant returns for investors. Microsoft is mentioned as one of the companies currently valued at over $2 trillion.\n   - [Read more](https://www.fool.com/investing/2024/08/08/prediction-1-stock-nvidia-microsoft-in-2-trillion/?source=iedfolrf0000001)\n\n2. **Nasdaq Correction: My Top "Magnificent Seven" Stock to Buy in August**\n   - **Publisher:** The Motley Fool\n   - **Published Date:** August 8, 2024\n   - **Summary:** The article discusses why Microsoft is a co

You can also ask about financial information for a company:

In [15]:
agent_executor.invoke(
    {"input": "What were last quarters financial numbers for Nvidia?"}
)



> Entering new AgentExecutor chain...

Invoking: `polygon_financials` with `{'query': 'NVDA'}`


[{"start_date": "2023-04-29", "end_date": "2024-04-28", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0001045810", "sic": "3674", "tickers": ["NVDA"], "company_name": "NVIDIA CORP", "financials": {"cash_flow_statement": {"net_cash_flow_from_operating_activities": {"value": 40524000000.0, "unit": "USD", "label": "Net Cash Flow From Operating Activities", "order": 100}, "net_cash_flow_from_financing_activities_continuing": {"value": -22598000000.0, "unit": "USD", "label": "Net Cash Flow From Financing Activities, Continuing", "order": 800}, "net_cash_flow_from_financing_activities": {"value": -22598000000.0, "unit": "USD", "label": "Net Cash Flow From Financing Activities", "order": 700}, "net_cash_flow": {"value": 2508000000.0, "unit": "USD", "label": "Net Cash Flow", "order": 1100}, "net_cash_flow_from_investing_activities": {"value": -15418000000.0, "unit": "USD"

{'input': 'What were last quarters financial numbers for Nvidia?',
 'output': "Here are the financial numbers for Nvidia for the last quarter (Q1 of fiscal year 2025):\n\n### Income Statement\n- **Revenues:** $26,044,000,000\n- **Cost of Revenue:** $5,638,000,000\n- **Gross Profit:** $20,406,000,000\n- **Operating Expenses:** $3,497,000,000\n  - **Research and Development:** $2,720,000,000\n  - **Selling, General, and Administrative Expenses:** $777,000,000\n- **Operating Income:** $16,909,000,000\n- **Nonoperating Income:** $370,000,000\n- **Income Before Tax:** $17,279,000,000\n- **Income Tax Expense:** $2,398,000,000\n- **Net Income:** $14,881,000,000\n- **Net Income Attributable to Parent:** $14,881,000,000\n- **Basic Earnings Per Share:** $6.04\n- **Diluted Earnings Per Share:** $5.98\n\n### Balance Sheet\n- **Total Assets:** $77,072,000,000\n  - **Current Assets:** $53,729,000,000\n    - **Inventory:** $5,864,000,000\n    - **Other Current Assets:** $47,865,000,000\n  - **Noncurr

Lastly, you can get live data, although this requires a "Stocks Advanced" subscription

In [ ]:
agent_executor.invoke({"input": "What is Doordash stock price right now?"})

### API reference

For detailed documentation of all the Polygon IO toolkit features and configurations head to the API reference: https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.polygon.toolkit.PolygonToolkit.html

## Tools

First, let's set up the API wrapper that we will use for all the tools and then we will walk through each one of them.

In [16]:
from langchain_community.utilities.polygon import PolygonAPIWrapper

api_wrapper = PolygonAPIWrapper()

### Aggregate

This tool shows aggregate information for a stock.

In [25]:
from langchain_community.tools.polygon.aggregates import PolygonAggregates

aggregate_tool = PolygonAggregates(api_wrapper=api_wrapper)

# We can invoke directly with input
res = aggregate_tool.invoke(
    {
        "ticker": "AAPL",
        "timespan": "day",
        "timespan_multiplier": 1,
        "from_date": "2024-08-01",
        "to_date": "2024-08-07",
    }
)

model_generated_tool_call = {
    "args": {
        "ticker": "AAPL",
        "timespan": "day",
        "timespan_multiplier": 1,
        "from_date": "2024-08-01",
        "to_date": "2024-08-07",
    },
    "id": "1",
    "name": aggregate_tool.name,
    "type": "tool_call",
}

# Or we can invoke with a tool call
res = aggregate_tool.invoke(model_generated_tool_call)

print(res)

content='[{"v": 61125243.0, "vw": 219.4773, "o": 224.37, "c": 218.36, "h": 224.48, "l": 217.02, "t": 1722484800000, "n": 876046}, {"v": 102635321.0, "vw": 221.9519, "o": 219.15, "c": 219.86, "h": 225.6, "l": 217.71, "t": 1722571200000, "n": 1132479}, {"v": 119382508.0, "vw": 207.801, "o": 199.09, "c": 209.27, "h": 213.5, "l": 196, "t": 1722830400000, "n": 1661493}, {"v": 69545387.0, "vw": 206.739, "o": 205.3, "c": 207.23, "h": 209.99, "l": 201.07, "t": 1722916800000, "n": 921946}, {"v": 60109650.0, "vw": 210.7375, "o": 206.9, "c": 209.82, "h": 213.64, "l": 206.39, "t": 1723003200000, "n": 741607}]' name='polygon_aggregates' tool_call_id='1'


### Financials

This tool provides general financial information about a stock

In [35]:
from langchain_community.tools.polygon.financials import PolygonFinancials

financials_tool = PolygonFinancials(api_wrapper=api_wrapper)

# We can invoke directly with input
res = financials_tool.invoke({"query": "AAPL"})

model_generated_tool_call = {
    "args": {"query": "AAPL"},
    "id": "1",
    "name": financials_tool.name,
    "type": "tool_call",
}

# Or we can invoke with a tool call
res = financials_tool.invoke(model_generated_tool_call)

print(res)

content='[{"start_date": "2023-03-31", "end_date": "2024-03-30", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0000320193", "sic": "3571", "tickers": ["AAPL"], "company_name": "APPLE INC", "financials": {"cash_flow_statement": {"net_cash_flow_from_operating_activities": {"value": 110563000000.0, "unit": "USD", "label": "Net Cash Flow From Operating Activities", "order": 100}, "net_cash_flow": {"value": 6792000000.0, "unit": "USD", "label": "Net Cash Flow", "order": 1100}, "net_cash_flow_from_investing_activities_continuing": {"value": 4448000000.0, "unit": "USD", "label": "Net Cash Flow From Investing Activities, Continuing", "order": 500}, "net_cash_flow_from_investing_activities": {"value": 4448000000.0, "unit": "USD", "label": "Net Cash Flow From Investing Activities", "order": 400}, "net_cash_flow_continuing": {"value": 6792000000.0, "unit": "USD", "label": "Net Cash Flow, Continuing", "order": 1200}, "net_cash_flow_from_operating_activities_continuing": {"

### Last Quote

This tool provides information about the live data of a stock, although it requires a "Stocks Advanced" subscription to use.

In [ ]:
from langchain_community.tools.polygon.last_quote import PolygonLastQuote

last_quote_tool = PolygonLastQuote(api_wrapper=api_wrapper)

# We can invoke directly with input
res = last_quote_tool.invoke({"query": "AAPL"})

model_generated_tool_call = {
    "args": {"query": "AAPL"},
    "id": "1",
    "name": last_quote_tool.name,
    "type": "tool_call",
}

# Or we can invoke with a tool call
res = last_quote_tool.invoke(model_generated_tool_call)

### Ticker News

This tool provides recent news about a certain ticker.

In [33]:
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews

news_tool = PolygonTickerNews(api_wrapper=api_wrapper)

# We can invoke directly with input
res = news_tool.invoke({"query": "AAPL"})

model_generated_tool_call = {
    "args": {"query": "AAPL"},
    "id": "1",
    "name": news_tool.name,
    "type": "tool_call",
}

# Or we can invoke with a tool call
res = news_tool.invoke(model_generated_tool_call)

print(res)

content='[{"id": "4fd9f3a653f3d1c6d99a21d7cc62b883e8547109c95778e34de7c6c9ecaccfe4", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "Warren Buffett Is Slashing Berkshire Hathaway\'s Position in Apple Stock", "author": "Parkev Tatevosian, Cfa", "published_utc": "2024-08-08T14:16:30Z", "article_url": "https://www.fool.com/investing/2024/08/08/warren-buffett-is-slashing-berkshire-hathaways-pos/?source=iedfolrf0000001", "tickers": ["AAPL"], "image_url": "https://g.foolcdn.com/editorial/images/786449/buffett17-tmf.jpg", "description": "Warren Buffett, the renowned investor, has sold a significant portion of his Apple stock holdings, worth around $90 billion. This move has raised questions about whether it could be Buffett\'s biggest investing mistake.", "keywords": ["Warren Buffett",

### API reference

For detailed documentation of all Polygon IO tools head to the API reference for each:

- Aggregate: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.polygon.aggregates.PolygonAggregates.html
- Financials: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.polygon.financials.PolygonFinancials.html
- Last Quote: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.polygon.last_quote.PolygonLastQuote.html
- Ticker News: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.polygon.ticker_news.PolygonTickerNews.html